<span style="font-size: 14pt">MIPT, Advanced ML, Autumn 2017</span>

<span style="font-size: 16pt"> HW #9: Recommendation System

<span style="color:blue; font-size: 12pt">Andrey Saitgalin </span>
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">andrey.saitgalin@gmail.com</span>, 

<span style="color:blue; font-size: 12pt">Alexey Dral </span>,
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">aadral@gmail.com</span>

<h1 align="center">Organization Info</h1> 

**Дополнительный материал для выполнения дз**:
<a href= "https://github.com/ml-mipt/ml-mipt-part2/tree/master/2017/lectures">Лекции по курсу</a>

**Оформление дз**: 
- Присылайте выполненное задание на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall <номер_группы> <фамилия>``, к примеру -- ``ML2017_Aut_fall 401 ivanov``
- Выполненное дз сохраните в файл ``<фамилия>_<группа>_task<номер>.ipnb``, к примеру -- ``ivanov_491_task9.ipnb``

**Вопросы**:
- Присылайте вопросы на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall Question <Содержание вопроса>``

** Дедлайн**:
<span style="color:red; font-size: 12pt">16.12.2017 23:59 </span>

--------
- **PS1**: Мы используем автоматические фильтры, и просто не найдем ваше дз, если вы не аккуратно его подпишите.
- **PS2**: Напоминаем, что дедлайны жесткие, письма пришедшие после автоматически удаляются =( чтобы соблазна не было.

<h1 align="center">Check Questions (0%)</h1> 
Вопросы отсутствуют

<h1 align="center"> Practical tasks (100%)</h1>


# Построение content-based рекомендательной системы
## Part 1
Для каждого фильма получите описание в текстовом виде с IMDB или (и) The movie database (у них есть открытое API). https://www.themoviedb.org/?language=ru. Вывести описание фильма под id 778. Также можно использовать рецензии.

In [2]:
import tmdbsimple as tmdb
tmdb.API_KEY = '8e3dcefbeebdf4e7f1776aca65359931'

In [5]:
movie = tmdb.Movies(778)

In [6]:
print(movie.info())

{'title': "Monsieur  Hulot's Holiday", 'overview': 'Monsieur Hulot, Jacques Tati’s endearing clown, takes a holiday at a seaside resort, where his presence provokes one catastrophe after another. Tati’s masterpiece of gentle slapstick is a series of effortlessly well-choreographed sight gags involving dogs, boats, and firecrackers; it was the first entry in the Hulot series and the film that launched its maker to international stardom.', 'release_date': '1953-02-26', 'status': 'Released', 'vote_count': 84, 'original_title': 'Les vacances de Monsieur Hulot', 'original_language': 'fr', 'belongs_to_collection': None, 'genres': [{'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}], 'budget': 0, 'id': 778, 'popularity': 4.765439, 'vote_average': 7.3, 'adult': False, 'video': False, 'poster_path': '/r4F4tsU0Ajeh9ZYUkWOJSYmioj7.jpg', 'backdrop_path': '/zF95knRgouUrBNbUh5oqRI5NIcE.jpg', 'revenue': 0, 'tagline': "It's laugh-vacation time as Jacques Tati romps through the most glorious

In [7]:
movie.info()['overview']

'Monsieur Hulot, Jacques Tati’s endearing clown, takes a holiday at a seaside resort, where his presence provokes one catastrophe after another. Tati’s masterpiece of gentle slapstick is a series of effortlessly well-choreographed sight gags involving dogs, boats, and firecrackers; it was the first entry in the Hulot series and the film that launched its maker to international stardom.'

## Part 2 
Извлеките текстовые признаки у фильмов: tf idf, биграммы (может какие-то еще). Биграммы - статистика по биграммам, сколько какая биграмма встречается. Формулы для TF-IDF смотреть тут: https://en.wikipedia.org/wiki/Tf%E2%80%93idf 
    1. Используйте TF с double 0.5 normalization
    2. Используйте стандартный IDF: 
$$idf(t, D) = \frac{log N}{(1 + |d in D : t in d|)}$$

In [8]:
if False:
    with open("movies_info.tsv", "w") as f:
        for i in range(2, 1000000000):
            try:
                movie = tmdb.Movies(idx)
                movie_info = movie.info()
                fields = [i, movie_info['title'], movie_info['overview'], movie_info['genres'], 
                          movie.reviews()['results']]
                filter_forbidden_chars = lambda s: str(s).replace('\t', ' ').replace('\n', ' ')
                s = ('\t'.join(map(filter_forbidden_chars, fields)))
                print(s)
                f.write(s + "\n")
            except:
                print("id not found", i)


In [9]:
import pandas as pd

In [29]:
tmdb_movies = pd.read_csv("movies_info.tsv", delimiter='\t', names=['id', 'title', 'overview', 'genres', 'reviews'], index_col=0)
tmdb_movies.head(2)

,title,overview,genres,reviews
id,,,,
2,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",[]
3,Shadows in Paradise,"An episode in the life of Nikander, a garbage ...","[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",[]


In [31]:
tmdb_movies.index = pd.to_numeric(tmdb_movies.index, errors='coerce')
tmdb_movies.dropna(inplace=True)

In [118]:
import ast
import numpy as np
import scipy
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import scipy.spatial.distance as ssd

In [40]:
movie_text = {}

def extract_genres(s):
    text = ""
    try:
        genres = ast.literal_eval(s)
        for g in genres:
            text = text + " " + g['name']
    except KeyboardInterrupt:
        raise
    except:
        pass
    return text

def extract_reviews(s):
    text = ""
    try:
        reviews = ast.literal_eval(s)
        for r in reviews:
            text = text + " " + r['content']
    except KeyboardInterrupt:
        raise
    except: 
        pass
    return text

for idx, row in tmdb_movies.iterrows():    
    
    text = " ".join([row.title, row.overview, extract_genres(row.genres), 
                     extract_reviews(row.reviews)])
    
    movie_text[int(idx)] = text



In [41]:
movie_text_items = movie_text.items()
movie_text_values = [x[1] for x in movie_text_items]

matrix_to_tmdb_id = {} # matrix ~ X
for i, val in enumerate(movie_text_items):
    matrix_to_tmdb_id[i] = val[0]
    

In [48]:
vectorizer = TfidfVectorizer(max_df=0.8, max_features=500,
                             min_df=0.01, stop_words='english',
                             use_idf=True)
Xs = vectorizer.fit_transform(movie_text_values).toarray()

X = {}

for i in range(len(Xs)):
    X[matrix_to_tmdb_id[i]] = Xs[i]

In [173]:
# фичи tfidf для фильма с id 92
X[92]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [58]:
count_vectorizer = CountVectorizer(stop_words='english', ngram_range=(2, 2))
bigrams_s = count_vectorizer.fit_transform(movie_id_text.values())

bigrams = {}

for i in range(len(Xs)):
    bigrams[matrix_to_tmdb_id[i]] = bigrams_s[i]

In [61]:
print(bigrams[2].shape)

(1, 547198)


In [55]:
count_vectorizer.vocabulary_

{'haddonfield town': 215987,
 'boy master': 56590,
 'bad roll': 37073,
 'daughters peter': 115814,
 'spy stumbles': 454951,
 'date best': 114948,
 'parody disney': 355733,
 'fell deeply': 174420,
 'enemy agents': 153065,
 'epic vs': 155944,
 '23 1945': 4243,
 'playing drifter': 369929,
 'ruthless gang': 416985,
 'powers transformation': 376286,
 'prize head': 380451,
 'ii bit': 239812,
 'daughter chaudhary': 115278,
 'son inside': 448849,
 'crabs make': 105624,
 'jorge begins': 259801,
 'proof pudding': 383249,
 'crib number': 107872,
 'mentored drama': 314334,
 'succeed frightening': 469000,
 'models salesmen': 321192,
 'interesting mix': 248548,
 'leaving aeroplane': 278559,
 'couple months': 104185,
 'helen shaver': 224005,
 'faced youngster': 165703,
 'brighton sunny': 59405,
 'pull nicely': 385822,
 'pretty ruins': 378578,
 'brings peace': 60060,
 'story prior': 463448,
 'suspect having': 474007,
 'music atmosphere': 329886,
 'safety innocent': 417997,
 'white bitch': 530397,
 'ho

In [62]:
bigrams[92].toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

In [63]:
bigrams[92].shape

(1, 547198)

In [64]:
bigrams[92].toarray()[0]

array([0, 0, 0, ..., 0, 0, 0])

## Part 3 
Выведите подсчитанные признаки из пункта 2 для фильма с id 99114 (нет такого, беру 100).

In [65]:
X[100]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.22250924,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.20863495,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [66]:
bigrams[100].toarray()[0]

array([0, 0, 0, ..., 0, 0, 0])

In [75]:
def key_by_val(d, value):
    for k, v in d.items():
        if v == value:
            return k

# встречающиеся в фильме биграммы
print([key_by_val(count_vectorizer.vocabulary_, i) for i, bigram_n in enumerate(bigrams[100].toarray()[0]) if bigram_n > 0])  

['barrels card', 'beautifully cast', 'brilliantly shot', 'card shark', 'cash quick', 'cast cleverly', 'cleverly produced', 'comedy crime', 'crime just', 'decide pull', 'door comedy', 'enlisted friends', 'far good', 'favorite movies', 'filmography genuinely', 'flat door', 'flaws pointing', 'friends need', 'gang happen', 'genuinely favorite', 'good movie', 'happen operating', 'heist small', 'just plain', 'lock stock', 'losing sketchy', 'lot cash', 'love movie', 'make lot', 'match decide', 'movie actually', 'movie far', 'movie tons', 'movies time', 'need make', 'operating flat', 'pattern ritchie', 'plain love', 'pointing pattern', 'poker match', 'produced movie', 'pull heist', 'quick losing', 'realised written', 'recently realised', 'ritchie filmography', 'shark unwillingly', 'shot beautifully', 'sketchy poker', 'small time', 'smoking barrels', 'stock smoking', 'time gang', 'time watched', 'tons flaws', 'unwillingly enlisted', 'watched recently', 'written brilliantly']


## Part 4
Для каждого пользователя найдите топ жанров, которые нравятся пользователю (можно делать как на семинаре top 5, top 3 и top 2) и порекомендуйте таким пользователя фильмы близкие по текстовому описанию (см. пункт 2). Для оценки результатов пункта 3 используйте метрику HitRate и метрики ранжирования (N)DCG. Формулу для подсчета HitRate можно взять на семинаре,  (N)DCG: https://en.wikipedia.org/wiki/Discounted_cumulative_gain



In [76]:
ml20_movie = pd.read_csv("ml-20m/movies.csv")

In [77]:
ml20_links = pd.read_csv("ml-20m/links.csv")
ml20_links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [82]:
movie_to_tmdb_id = {}
tmdb_to_movie_id = {}

problem_idxs = set()

for idx, row in ml20_links.iterrows():
    try:
        movie_to_tmdb_id[int(row.movieId)] = int(row.tmdbId)
        tmdb_to_movie_id[int(row.tmdbId)] = int(row.movieId)
    except:
        problem_idxs.add(idx)
        #print("Problem in idx", idx, row)
        
print(len(problem_idxs))

252


In [102]:
display(ml20_links.tail(3))
display(ml20_movie.head(2))

,movieId,imdbId,tmdbId
27275,131258,3485166,285213.0
27276,131260,249110,32099.0
27277,131262,1724965,286971.0


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [86]:
ml20_ratings = pd.read_csv("ml-20m/ratings.csv")
ml20_ratings.head(2)

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676


In [87]:
# copy paste from seminar

In [92]:
from collections import Counter

genres = []

for row in ml20_movie.itertuples():
    genre_list = row.genres.split("|")
    genres.extend(genre_list)

cntr = Counter(genres)
display(cntr)
display(cntr.most_common(5))

Counter({'(no genres listed)': 246,
         'Action': 3520,
         'Adventure': 2329,
         'Animation': 1027,
         'Children': 1139,
         'Comedy': 8374,
         'Crime': 2939,
         'Documentary': 2471,
         'Drama': 13344,
         'Fantasy': 1412,
         'Film-Noir': 330,
         'Horror': 2611,
         'IMAX': 196,
         'Musical': 1036,
         'Mystery': 1514,
         'Romance': 4127,
         'Sci-Fi': 1743,
         'Thriller': 4178,
         'War': 1194,
         'Western': 676})

[('Drama', 13344),
 ('Comedy', 8374),
 ('Thriller', 4178),
 ('Romance', 4127),
 ('Action', 3520)]

In [96]:
ml20_joined = pd.merge(ml20_ratings, ml20_movie[["movieId", "genres"]], on="movieId", how="inner")
ml20_joined.head()

,userId,movieId,rating,timestamp,genres
0,1,2,3.5,1112486027,Adventure|Children|Fantasy
1,5,2,3.0,851527569,Adventure|Children|Fantasy
2,13,2,3.0,849082742,Adventure|Children|Fantasy
3,29,2,3.0,835562174,Adventure|Children|Fantasy
4,34,2,3.0,846509384,Adventure|Children|Fantasy


In [97]:
def popular_genres(x):
    return Counter("|".join(x.tolist()).split("|"))

def popular_genres_top5(x):
    return popular_genres(x).most_common(5)

ml20_users = ml20_joined.groupby('userId').agg({'genres': [popular_genres_top5]})


In [99]:
ml20_users.head(2)

,genres
,popular_genres_top5
userId,
1,"[(Adventure, 73), (Fantasy, 69), (Action, 66),..."
2,"[(Sci-Fi, 23), (Thriller, 19), (Action, 19), (..."


In [100]:
ml20_users.loc[1]

genres  popular_genres_top5    [(Adventure, 73), (Fantasy, 69), (Action, 66),...
Name: 1, dtype: object

In [107]:
display(ml20_users.loc[1]['genres']['popular_genres_top5'])
display(ml20_users.loc[2]['genres']['popular_genres_top5'])

[('Adventure', 73),
 ('Fantasy', 69),
 ('Action', 66),
 ('Horror', 45),
 ('Drama', 43)]

[('Sci-Fi', 23),
 ('Thriller', 19),
 ('Action', 19),
 ('Drama', 19),
 ('Horror', 18)]

In [113]:
def fast_top_genres(user_id, top_count):
    return ml20_users.loc[user_id]['genres']['popular_genres_top5'][:top_count]

In [114]:
fast_top_genres(user_id=100, top_count=4)

[('Drama', 26), ('Comedy', 24), ('Thriller', 16), ('Action', 12)]

In [116]:
fast_top_genres(2017, 3)

[('Comedy', 16), ('Action', 13), ('Drama', 13)]

In [143]:
vector_len = lambda v : (v * v.T).sum() ** 0.5

def calculate_likeness(rated_movies, movie):
    likeness = 0
    for rated_id, rating, tfidf, bigram, bigram_len in rated_movies:
        try:
            likeness = likeness + \
                       (ssd.cosine(tfidf, movie[1]) + \
                        (bigram * movie[2].T).sum() / bigram_len / movie[3]) * (rating - 3)
        except KeyboardInterrupt:
            raise
        except:
            raise
            pass
        
    return likeness
    

def calculate_likeness_list(user_id):
    ml20_ratings_u = ml20_ratings[ml20_ratings.userId == user_id]
    
    rated_movies = []  # (movieId, rating)
    for idx, row in ml20_ratings_u.iterrows():
        try:
            tmdb_id = movie_to_tmdb_id[int(row.movieId)]
            rating = row.rating
            tfidf = X[tmdb_id]
            bigram = bigrams[tmdb_id]
            rated_movies.append((tmdb_id, row.rating, tfidf, bigram, vector_len(bigram)))
        except:
            pass
    
    # backup
    with open("likeness.tsv", 'w') as f:
        idx_to_likeness_list = []
        for idx, row in tmdb_movies.iterrows():
            try:
                tfidf = X[idx]
                bigram = bigrams[idx]
                likeness = calculate_likeness(rated_movies, (idx, tfidf, bigram, vector_len(bigram)))
                idx_to_likeness_list.append((idx, likeness))
                f.write(str(idx) + "\t" + str(likeness) + "\n")
                f.flush()
            except:
                print("problem with", idx)
                raise
        
    return idx_to_likeness_list
    

In [ ]:
likeness_list = calculate_likeness_list(2)

In [171]:
likeness_list_fixed = list(map(lambda x: (int(x[0]), x[1]), likeness_list))
top = sorted(likeness_list_fixed, key=lambda x:x[1])[:10]
top

[(609, 107.00727754),
 (10195, 107.19161189399999),
 (10140, 107.21920309399999),
 (578, 107.53372634899999),
 (24418, 107.99211289799999),
 (98, 108.49449593600001),
 (10764, 108.55491052799999),
 (630, 108.68208507999999),
 (11906, 108.78268886399999),
 (1895, 108.820072834)]

In [172]:
# рекоммендации для пользователя 2
[tmdb_movies[tmdb_movies.index == tmdb_id].title for tmdb_id, likeness in top]

[id
 609.0    Poltergeist
 Name: title, dtype: object, id
 10195.0    Thor
 Name: title, dtype: object, id
 10140.0    The Chronicles of Narnia: The Voyage of the Da...
 Name: title, dtype: object, id
 578.0    Jaws
 Name: title, dtype: object, id
 24418.0    Cirque du Freak: The Vampire's Assistant
 Name: title, dtype: object, id
 98.0    Gladiator
 Name: title, dtype: object, id
 10764.0    Quantum of Solace
 Name: title, dtype: object, id
 630.0    The Wizard of Oz
 Name: title, dtype: object, id
 11906.0    Suspiria
 Name: title, dtype: object, id
 1895.0    Star Wars: Episode III - Revenge of the Sith
 Name: title, dtype: object]

Нет времени делать оценку качества, увы(